Project Report for MAP543 - Database Management II  
Authors: Roberta Conrad, Dora Ranilovic, Zigfrid Zvezdin  
dependecies: Python 3.6.4+

# TF IDF Report

## Table of Content 

1. Map Reduce Algorithm: a description of the _adopted solution_

    1a. Other designed algorithms plus related global comments/description including comments to the code 

3. Experimental analysis, concerning in particular scalability 

    2a. Comments about the experimental analysis outlining weak and strong points of the algorithms.

4. an appendix including all the code

# 1. MapReduce algorithm
calculate TF-IDF scores given an input set of documents. Provide both a MapReduce Python Hadoop streaming and a Spark implementation. 
## 1.a. Adopted Version

### Python Hadoop

The first implementation of Tf-Idf is a tradidional MapReduce algorithm, designed to be run on a Hadoop cluster with only Python 2.7/3.+ dependencies. It consists of consecutive Map and Reduce jobs, with the following inputs and outputs:

### Spark

## 1.b. Other designed algorithms

### Python Hadoop

### Spark


# 2. Experimental analysis

In order to compare the performance of the three different implementations, we designed two different tests considering different structures of a text collection with increasing sizes. We differentiated between a collection of documents containing many short documents; and a collection containing few long documents:

+ Collections with a fixed document size: we fixed the number of words at 100 words per document, changing the number of documents in (100,200,300,400,500,1000).  
+ Collections with a fixed number of documents: We ran our tf-idf algorithms on 100 documents containing (100,200,300,400,500,1000) words. 

For easy reproducability this report contains all code and scripts that have been used to produce the following analysis and can be recreated on AWS Elastic Map Reduce cluster. 

*Limitations*  
In order to control the size of the documents used to test our algorithm, we decided to write a script creating documents for testing purposes. One document containing 100 words has a size of 11 KB when created by our script. 

## 2.1. Comparison of the implementations

The second Spark implementation outperforms the other algorithms for almost all input sizes and scales as document size and/or number increase. We prefer the second Spark algorithm for both a few, long documents, as well as many very short ones - such as tweets or emails, for example. 

The first Spark algorithm is the most efficient algorithm for a very small text collection. However, it does not scale with size and turns out to be very slow for increasingly larger document sizes, while it shows the exact same performance as the fist Spark implementation for many small documents. 

The MapReduce implementation is the slowest option of all three and only better then the first Spark implementation as documents become longer then 300 Words (approx. 30 KB). 

<img src="./longdoc.PNG">

<img src="./Manydoc.PNG">


### 2.2. Practical Instructions to run tests on the cluster

1. Create text documents on hadoop home directory  
    + create the input directory on /user/hadoop local file system 
    + in /input, import the python script and execute the text_generator function to create documents needed, specify params as described below. 
    + move only the .txt to the hdfs hadoop input directory 

2. Download and run the MapReduce algorithms
 + give permission to access all documents
 + execute all jobs through bash script runjobs.sh
 + after retrieving the time information, clear input directories with clear.sh

# 3. Code Appendix

### text_generator.py
*Requirements* Requires installing the nlkt library brown.   

*Functionality* The function create_docs takes as input parameters the number of words per document, number of documents, and start number of the document to be taken into account when naming files. The function creates the specified number of documents at the specified length by taking samples of the brown corpus. It randomly samples lines of twenty words at the same time.

The Brown University Standard Corpus of Present-Day American English (or just Brown Corpus) was compiled in the 1960s as a general corpus. It contains 500 samples of English-language text, totaling roughly one million words, compiled from works published in the United States in 1961. It is a suitable library to sample from in order to test our algorithm as it represents how language is used in reality. 

In [ ]:
#!/usr/bin/python
from nltk.corpus import brown
import random
corpus_length = len(brown.words())
hardcopy = brown.words()


def create_docs(number_of_words_per_doc=200, num_doc=10, startnr=0):
    # control number of words per doc
    # and number of documents
    # we fix the line length at 20
    line_length = 20
    number_of_lines = int(number_of_words_per_doc / line_length)

    for i in range(0, num_doc):
        # create new file with writing + permission
        new_file = open("textdoc"+str(number_of_words_per_doc) +
                        "words" + str(i+startnr)+".txt", "w+")
        for line in range(0, number_of_lines):
            words = list(map(
                lambda x: hardcopy[x:x+line_length], random.sample(range(corpus_length), line_length)))
            sentences = list(
                map(lambda x: ' '.join(word for word in x), words))
            text = ''.join(map(str, sentences))
            new_file.write(text + "\n")
        new_file.close()
        if (i % 10 == 0):
            print("You created "+str(i)+" files! "+str(num_doc-i)+" left")